In [1]:
from datasets import load_dataset

In [2]:
mnli = load_dataset("glue", "mnli")

Using the latest cached version of the dataset since glue couldn't be found on the Hugging Face Hub
Found the latest cached dataset configuration 'mnli' at /home/mtheologitis/.cache/huggingface/datasets/glue/mnli/0.0.0/bcdcba79d07bc864c1c254ccfcedcce55bcc9a8c (last modified on Thu Jul 25 14:20:10 2024).


In [38]:
mnli['train'].features

{'premise': Value(dtype='string', id=None),
 'hypothesis': Value(dtype='string', id=None),
 'label': ClassLabel(names=['entailment', 'neutral', 'contradiction'], id=None),
 'idx': Value(dtype='int32', id=None)}

In [39]:
mnli['train'].num_rows

392702

In [20]:
qqp = load_dataset("glue", "qqp")

In [40]:
qqp['train'].features

{'question1': Value(dtype='string', id=None),
 'question2': Value(dtype='string', id=None),
 'label': ClassLabel(names=['not_duplicate', 'duplicate'], id=None),
 'idx': Value(dtype='int32', id=None)}

In [41]:
qqp['train'].num_rows

363846

In [21]:
qnli = load_dataset("glue", "qnli")

In [42]:
qnli['train'].features

{'question': Value(dtype='string', id=None),
 'sentence': Value(dtype='string', id=None),
 'label': ClassLabel(names=['entailment', 'not_entailment'], id=None),
 'idx': Value(dtype='int32', id=None)}

In [43]:
qnli['train'].num_rows

104743

In [30]:
104743 / (1000 * 8)

13.092875

In [6]:
sst2 = load_dataset("glue", "sst2")

Using the latest cached version of the dataset since glue couldn't be found on the Hugging Face Hub
Found the latest cached dataset configuration 'sst2' at /home/mtheologitis/.cache/huggingface/datasets/glue/sst2/0.0.0/bcdcba79d07bc864c1c254ccfcedcce55bcc9a8c (last modified on Thu Jul 25 13:59:21 2024).


In [7]:
sst2['train'].features

{'sentence': Value(dtype='string', id=None),
 'label': ClassLabel(names=['negative', 'positive'], id=None),
 'idx': Value(dtype='int32', id=None)}

In [45]:
sst2['train'].num_rows

67349

In [5]:
67349 / (100 * 8)

84.18625

In [23]:
cola = load_dataset("glue", "cola")

In [46]:
cola['train'].features

{'sentence': Value(dtype='string', id=None),
 'label': ClassLabel(names=['unacceptable', 'acceptable'], id=None),
 'idx': Value(dtype='int32', id=None)}

In [47]:
cola['train'].num_rows

8551

In [10]:
8551 / (50 * 8)

21.3775

In [3]:
stsb = load_dataset("glue", "stsb")

Using the latest cached version of the dataset since glue couldn't be found on the Hugging Face Hub
Found the latest cached dataset configuration 'stsb' at /home/mtheologitis/.cache/huggingface/datasets/glue/stsb/0.0.0/bcdcba79d07bc864c1c254ccfcedcce55bcc9a8c (last modified on Tue Dec 17 11:36:27 2024).


In [5]:
stsb['train'].features

{'sentence1': Value(dtype='string', id=None),
 'sentence2': Value(dtype='string', id=None),
 'label': Value(dtype='float32', id=None),
 'idx': Value(dtype='int32', id=None)}

In [6]:
stsb['train'].num_rows

5749

In [15]:
mrpc = load_dataset("glue", "mrpc")

Using the latest cached version of the dataset since glue couldn't be found on the Hugging Face Hub
Found the latest cached dataset configuration 'mrpc' at /home/mtheologitis/.cache/huggingface/datasets/glue/mrpc/0.0.0/bcdcba79d07bc864c1c254ccfcedcce55bcc9a8c (last modified on Tue Dec 17 12:39:27 2024).


In [16]:
mrpc['train'].features

{'sentence1': Value(dtype='string', id=None),
 'sentence2': Value(dtype='string', id=None),
 'label': ClassLabel(names=['not_equivalent', 'equivalent'], id=None),
 'idx': Value(dtype='int32', id=None)}

In [51]:
mrpc['train'].num_rows

3668

In [20]:
mrpc['train'].features['label'].num_classes

2

In [26]:
rte = load_dataset("glue", "rte")

In [37]:
rte['train'].features

{'sentence1': Value(dtype='string', id=None),
 'sentence2': Value(dtype='string', id=None),
 'label': ClassLabel(names=['entailment', 'not_entailment'], id=None),
 'idx': Value(dtype='int32', id=None)}

In [52]:
rte

DatasetDict({
    train: Dataset({
        features: ['sentence1', 'sentence2', 'label', 'idx'],
        num_rows: 2490
    })
    validation: Dataset({
        features: ['sentence1', 'sentence2', 'label', 'idx'],
        num_rows: 277
    })
    test: Dataset({
        features: ['sentence1', 'sentence2', 'label', 'idx'],
        num_rows: 3000
    })
})

In [34]:
rte['train'].num_rows

2490

In [60]:
2490 / (10 * 8)

31.125